# Pseudo-Code on the Byte-Pair Encoding algorithm
- Reference Article: [Neural Machine Translation of Rare Words with Subword Units](https://aclanthology.org/P16-1162/)
- Code Generated with **Claude 3.7**

In [1]:
# Sample corpus
corpus = [
    "the cat sat on the mat",
    "the dog sat on the log",
    "cats and dogs are friends",
]

In [2]:
from collections import defaultdict, Counter
# Count the frequency of each word in the corpus
def get_word_frequencies(corpus):
    word_freqs = Counter()
    for sentence in corpus:
        for word in sentence.strip().split():
            word_freqs[word] += 1
    return word_freqs
word_freqs = get_word_frequencies(corpus)
print(word_freqs)

Counter({'the': 4, 'sat': 2, 'on': 2, 'cat': 1, 'mat': 1, 'dog': 1, 'log': 1, 'cats': 1, 'and': 1, 'dogs': 1, 'are': 1, 'friends': 1})


In [3]:
import string
# Remember: BPE starts from characters!
def initialize_vocab_with_alphabet():
    """Initialize vocabulary with all lowercase English alphabet characters"""
    # Include lowercase letters, digits, and common punctuation
    vocab = set(string.ascii_lowercase + string.digits + string.punctuation + ' ')
    return vocab
initial_vocab = initialize_vocab_with_alphabet()
print(f"Initial vocab: {sorted(initial_vocab)}")

Initial vocab: [' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~']


In [4]:
# All words in the initial corpus are initially tokenized as single characters
splits = {word: list(word) for word in word_freqs.keys()}
print(f"How words are currently split:")
splits

How words are currently split:


{'the': ['t', 'h', 'e'],
 'cat': ['c', 'a', 't'],
 'sat': ['s', 'a', 't'],
 'on': ['o', 'n'],
 'mat': ['m', 'a', 't'],
 'dog': ['d', 'o', 'g'],
 'log': ['l', 'o', 'g'],
 'cats': ['c', 'a', 't', 's'],
 'and': ['a', 'n', 'd'],
 'dogs': ['d', 'o', 'g', 's'],
 'are': ['a', 'r', 'e'],
 'friends': ['f', 'r', 'i', 'e', 'n', 'd', 's']}

In [5]:
# To choose the most frequent pair we want to join, we must take into account of the word frequencies in the corpus
def estimate_token_frequencies(word_freqs, current_splits):
    pair_freqs = defaultdict(int)
    # For each word and associated frequency
    for word, freq in word_freqs.items():
        # Recover how that word is currently split into tokens
        current_split = current_splits[word]
        # Keep track of how frequent each combination of subsequent letter for that word is
        for j in range(len(current_split) - 1):
            pair = (current_split[j], current_split[j + 1])
            pair_freqs[pair] += freq
    # Find the most frequent pair
    best_pair, occurrences = max(pair_freqs.items(), key=lambda x: x[1])
    return best_pair, occurrences
best_pair, occurrences = estimate_token_frequencies(word_freqs, splits)
print(f"The most frequent pair of character is {best_pair} -> {occurrences} occurrences")

The most frequent pair of character is ('a', 't') -> 5 occurrences


In [6]:
# Create new merged token and add to vocabulary
new_token = ''.join(best_pair)
vocab = {value for value in initial_vocab} # working copy of initial vocab
vocab.add(new_token)
print(f"Updated vocab: {sorted(vocab)}")

Updated vocab: [' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', 'a', 'at', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~']


In [7]:
# Now, update the way each word is split into tokens according to the new merge!
def update_word_splits(best_pair, word_freqs, splits):
    for word in word_freqs.keys():
        split = splits[word]
        # Apply the merge to this word
        new_split = []
        i = 0
        while i < len(split):
            if i < len(split) - 1 and (split[i], split[i + 1]) == best_pair:
                new_split.append(new_token)
                i += 2
            else:
                new_split.append(split[i])
                i += 1
        # Update the split for this word
        splits[word] = new_split
    return splits
updated_splits = update_word_splits(best_pair, word_freqs, splits)
print(f"How words are split after the first merge:")
updated_splits

How words are split after the first merge:


{'the': ['t', 'h', 'e'],
 'cat': ['c', 'at'],
 'sat': ['s', 'at'],
 'on': ['o', 'n'],
 'mat': ['m', 'at'],
 'dog': ['d', 'o', 'g'],
 'log': ['l', 'o', 'g'],
 'cats': ['c', 'at', 's'],
 'and': ['a', 'n', 'd'],
 'dogs': ['d', 'o', 'g', 's'],
 'are': ['a', 'r', 'e'],
 'friends': ['f', 'r', 'i', 'e', 'n', 'd', 's']}

In [8]:
# For BPE, simply repeat for num_merges!
def train_bpe(corpus, num_merges):
    """Train a BPE tokenizer on the corpus"""
    # Start by splitting sentences into words and count their frequencies
    word_freqs = get_word_frequencies(corpus)
    # Initialize the vocabulary with individual characters
    vocab = initialize_vocab_with_alphabet()
    print(f"\tBase vocabulary size (unique characters): {len(vocab)}")
    print(f"\tBase vocabulary: {sorted(vocab)}")
    # Initialize the list of merge operations
    merges = {}
    # Split words into characters 
    splits = {word: list(word) for word in word_freqs.keys()}
    # Perform the specified number of merges
    for i in range(num_merges):
        # Select best pair, given current splits
        best_pair, occurrences = estimate_token_frequencies(word_freqs, splits)
        # Create new merged token and add to vocabulary
        new_token = ''.join(best_pair)
        vocab.add(new_token)
        merges[best_pair] = new_token
        print(f"\tMerge {i+1}: {best_pair} -> {new_token} (frequency: {occurrences})")
        splits = update_word_splits(best_pair, word_freqs, splits)    
    # Calculate final vocabulary size
    print(f"\t\t- Base characters: {len(vocab) - len(merges)}")
    print(f"\t\t- Merged tokens: {len(merges)}")
    print(f"\t\t- Total vocabulary size: {len(vocab)}")
    # Return the vocabulary and merges for tokenization
    return vocab, merges

# Train BPE
print(f"Training BPE tokenizer starting from corpus: {corpus}")
vocab, merges = train_bpe(corpus, num_merges=10)
print(f"\nFinal vocabulary: {sorted(vocab)}")
print(f"\nFinal merges: {merges}")

Training BPE tokenizer starting from corpus: ['the cat sat on the mat', 'the dog sat on the log', 'cats and dogs are friends']
	Base vocabulary size (unique characters): 69
	Base vocabulary: [' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~']
	Merge 1: ('a', 't') -> at (frequency: 5)
	Merge 2: ('t', 'h') -> th (frequency: 4)
	Merge 3: ('at', 'e') -> ate (frequency: 4)
	Merge 4: ('o', 'g') -> og (frequency: 3)
	Merge 5: ('c', 'at') -> cat (frequency: 2)
	Merge 6: ('s', 'at') -> sat (frequency: 2)
	Merge 7: ('o', 'n') -> on (frequency: 2)
	Merge 8: ('d', 'at') -> dat (frequency: 2)
	Merge 9: ('at', 's') -> ats (frequency: 2)
	Merge 10: ('n', 'd') -> nd (frequency: 2)
		- Base characters: 69
		- Mer

### Examples of tokenization using the trained algorithm

In [9]:
def tokenize(text, merges):
    """Tokenize text using learned merges"""
    # Start with character-level tokenization
    tokens = list(text)
    # Apply merges in order
    for (first, second), merged in merges.items():
        i = 0
        while i < len(tokens) - 1:
            if tokens[i] == first and tokens[i + 1] == second:
                tokens[i] = merged
                del tokens[i + 1]
            else:
                i += 1
    return tokens

In [10]:
print("Tokenization Examples:")
test_words = ["the", "cat", "sat", "cats", "friends", "catsat"]
for word in test_words:
    tokens = tokenize(word, merges)
    print(f"'{word}' -> {tokens}")

Tokenization Examples:
'the' -> ['th', 'e']
'cat' -> ['cat']
'sat' -> ['sat']
'cats' -> ['cat', 's']
'friends' -> ['f', 'r', 'i', 'e', 'nd', 's']
'catsat' -> ['cat', 'sat']


In [11]:
print("With a full sentence:")
test_text = "the cat sat on the mat with friends"
tokens = []
for word in test_text.split():
    tokens.extend(tokenize(word, merges))
    
print(f"\nFull text tokenization:")
print(f"'{test_text}' -> {tokens}")

With a full sentence:

Full text tokenization:
'the cat sat on the mat with friends' -> ['th', 'e', 'cat', 'sat', 'on', 'th', 'e', 'm', 'at', 'w', 'i', 'th', 'f', 'r', 'i', 'e', 'nd', 's']
